### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [4]:
# --- Multimodal chat (imagen + texto) en OCI Generative AI---

import os, json, base64, mimetypes
import oci

# === Configuración base ===
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file(os.path.expanduser("~/.oci/config"), CONFIG_PROFILE)
config["region"] = "us-chicago-1"

COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaawy5gpq67r3rxc7yf6yn7oesvvgr56dqg7o5y3f5ndzt2mddtj46a"
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

MODEL_ID = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyayjawvuonfkw2ua4bob4rlnnlhs522pafbglivtwlfzta"
assert ".oc1.us-chicago-1." in MODEL_ID, "El model_id debe ser de la región us-chicago-1"

# === Utilidad: convertir imagen local a data URL base64 ===
def image_to_data_url(path: str) -> str:
    mime = mimetypes.guess_type(path)[0] or "image/jpeg"
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:{mime};base64,{b64}"

# Ruta de tu imagen local
IMG_PATH = "demo-image.jpeg"
PROMPT = "Describe brevemente lo que ves en la imagen."

# Prepara contenidos multimodales
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import (
    ChatDetails,
    OnDemandServingMode,
    GenericChatRequest,
    BaseChatRequest,
    Message,
    TextContent,
    ImageContent,
    ImageUrl,
)

# Imagen como data URL (también puedes usar una URL pública HTTPS)
img_data_url = image_to_data_url(IMG_PATH)

image_url = ImageUrl(url=img_data_url)
image_content = ImageContent(image_url=image_url)

text_content = TextContent(text=PROMPT)

user_message = Message(role="USER", content=[text_content, image_content])

chat_request = GenericChatRequest(
    api_format=BaseChatRequest.API_FORMAT_GENERIC,
    messages=[user_message],
    max_tokens=400,
    temperature=0.4,
    top_p=0.9,
)

chat_details = ChatDetails(
    serving_mode=OnDemandServingMode(model_id=MODEL_ID),
    chat_request=chat_request,
    compartment_id=COMPARTMENT_ID,
)

client = GenerativeAiInferenceClient(config=config, service_endpoint=ENDPOINT)
resp = client.chat(chat_details)

# Extrae y muestra el texto de la respuesta
choices = resp.data.chat_response.choices
if choices:
    parts = choices[0].message.content
    texto = "\n".join(getattr(p, "text", "") for p in parts if hasattr(p, "text"))
    print("\n=== RESPUESTA ===\n", texto.strip())
else:
    print("No se generó respuesta.")


=== RESPUESTA ===
 La imagen muestra a una mujer tomando una selfie con una llama en un entorno al aire libre. La mujer, con el pelo oscuro y rizado y gafas negras, sonríe y mira directamente a la cámara. Lleva una chaqueta rosa sobre una camiseta azul oscura. La llama, con un abrigo blanco y manchas marrones, se para a su derecha, mirando hacia la cámara con las orejas hacia arriba. En el fondo, un camino de tierra conduce a una valla de madera, con árboles y una casa visible a lo lejos. La atmósfera general sugiere un día soleado en un zoológico o santuario de vida silvestre.
